In [9]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta


cdc_vaccine_data = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\CDC US State Vaccinations Processed.xlsx')
state_map = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\State abbreviation to Full Names.xlsx')
vaccine_data = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\OWID Global Vax Processed.xlsx')

us_vaccine_data = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\OCT_CDC US State Vaccinations Processed_OD_SUM_SafetoShare.xlsx')


def usa_total_vaccine(df):
    
    filter_cond = (df['Latest_Date_Rank'] == 1) & (df['location'] == "United States")
    
    filtered_vaccine_data = df[filter_cond]
    
    usa_total_vaccination = round(filtered_vaccine_data['total_vaccinations'].sum())
    
    maxmonth = filtered_vaccine_data['date'].max()
    
    #print(type(maxmonth))
    month = maxmonth.month
    day =maxmonth.day
    
    a = '{:,.0f} total vaccine doses administered as of {}/{}.'.format(usa_total_vaccination,month,day) 
    
    return a


def global_total_vaccine(df):
    
    filter_cond = (df['Latest_Date_Rank'] == 1) & (df['location'] == "World")
    
    filtered_vaccine_data = df[filter_cond]
    
    global_total_vaccination = round(filtered_vaccine_data['total_vaccinations'].sum())
    
    maxmonth = filtered_vaccine_data['date'].max()
    
    month = maxmonth.month
    day =maxmonth.day
    
    #month = df[df['location'] == "World"]['date'].dt.month.item()
    #day = df[df['location'] == "World"]['date'].dt.day.item()

    loc_cond_filter = (df['Latest_Date_Rank'] == 1) & (~df['location'].isin(['England','Wales','World','Northern Ireland','Northern Cyprus','European Union','Scotland']))
    #loc_cond_filter2 = (df['location'].isin(['England','Wales','World','Northern Ireland','Northern Cyprus','European Union','Scotland']))
    
    
    loc_cnt = df[loc_cond_filter]['location'].count()
 
    a = '{:,} total vaccine doses administered across {} countries as of {}/{}.'.format(global_total_vaccination,loc_cnt,month,day) 
    
    return a


def global_7day_ma(df):
    
    filter_cond = (df['Latest_Date_Rank'] == 1) & (df['location'] == "World")
    
    filter_cond_pm = (df['Latest_Date_Rank'] == 29) & (df['location'] == "World")
    
    filtered_7day_ma_data = df[filter_cond]
    
    filtered_7day_ma_data_pm = df[filter_cond_pm]

    global_7day_ma = round(filtered_7day_ma_data['daily_vaccinations 7 Day MA'].sum())
    
    global_7day_ma_pm = round(filtered_7day_ma_data_pm['daily_vaccinations 7 Day MA'].sum())
    
    try:
        change_per =(global_7day_ma/global_7day_ma_pm - 1)*100
    except:
        return 0
    
    maxdt_cw = filtered_7day_ma_data['date'].max()
       
    month_cw = maxdt_cw.month
    day_cw =maxdt_cw.day

    if(change_per > 0):
        flag = 'up'
    else:
        flag = 'down'
    
    a = '{:,.0f} nominal 7-day moving average of vaccine doses administered on {}/{}, {} {:.1f}% from {:,.0f} one month ago.'.format(global_7day_ma,month_cw,day_cw,flag,abs(change_per),global_7day_ma_pm) 
    
    return a

def global_uhg_loc_vax(df):
    
    filter_cond = (df['UHG Country'] == "UHG") & (df['location'] != "World") & (df['Latest_Date_Rank'] == 1)
    
    filtered_vaccine_data = df[filter_cond]
    
    vaccine_country_lst = filtered_vaccine_data['location'].iloc[0:50].tolist()
    vaccine_country_ppl_vax = filtered_vaccine_data['people_vaccinated_per_hundred'].iloc[0:50].tolist()
    vaccine_country_tot_vax = filtered_vaccine_data['total_vaccinations_per_hundred'].iloc[0:50].tolist()
    vaccine_country_ppl_cnt = filtered_vaccine_data['people_vaccinated'].iloc[0:50].tolist()
    vaccine_country_tot_cnt = filtered_vaccine_data['total_vaccinations'].iloc[0:50].tolist()

    vaccine_country = [item.replace('United Kingdom', 'UK') for item in vaccine_country_lst]
    vaccine_country = [item.replace('United States', 'U.S.') for item in vaccine_country]
            

    vaccine_country_dict = {}
    country_vaccine_count_dict ={}

    for i in range(0,len(vaccine_country)):
        if(np.isnan(vaccine_country_ppl_vax[i]) or np.isnan(vaccine_country_ppl_cnt[i])):
            vaccine_country_dict[vaccine_country[i]] = [vaccine_country_tot_vax[i],vaccine_country_tot_cnt[i]]
        else:
            vaccine_country_dict[vaccine_country[i]] = [vaccine_country_ppl_vax[i],vaccine_country_ppl_cnt[i]]
        
    vaccine_country_dict = {k: v for k, v in sorted(vaccine_country_dict.items(),reverse = True, key=lambda x: x[1])} 

    expression = []
    
    for i in range(0, len(vaccine_country_dict),1):
        expression.append(list(vaccine_country_dict)[i])
        expression.append((list(vaccine_country_dict.values())[i][0]))
        expression.append((list(vaccine_country_dict.values())[i][1]))
        
    
    if len(vaccine_country_dict) == 0:
          a = ''
    
    elif len(vaccine_country_dict) == 1:
      
        a = 'The key UHG locations with the highest percentage of population vaccinated are: {} {:.1f}% ({:,.0f}).'.format(*expression)
        
    
    elif len(vaccine_country_dict) == 2:
        
        a = 'The key UHG locations with the highest percentage of population vaccinated are: {} {:.1f}% ({:,.0f}) and {} ({:.1f}%) ({:,.0f})'.format(*expression)
        
    elif len(vaccine_country_dict) > 2:
        
        comment_length = len(vaccine_country_dict) - 2
        comment = 'The key UHG locations with the highest percentage of population vaccinated are: ' + '{} {:.1f}% ({:,.0f})' + comment_length*(', {} {:.1f}% ({:,.0f})') + ' and {} {:.1f}% ({:,.0f})' + str("") + '.'.format(*expression)
        a = comment.format(*expression)
    return a
  
def us_uhg_state_vaccines(df,statedf):
    
    statedf['location'] = statedf['Name']
    
    df = df.merge(statedf, how = 'inner', on = ['location'])
   
    filter_cond = (df['Latest_Date_Rank'] == 1) & (df['location'].isin(['Arizona','California','Connecticut','Florida','Minnesota','Nevada','New York','Pennsylvania','Texas','Wisconsin']))
      
    filter_us_uhg_state_vaccine_data = df[filter_cond]

    vaccine_state = filter_us_uhg_state_vaccine_data['State or Region Code'].iloc[0:50].tolist()
    vaccine_state_values = filter_us_uhg_state_vaccine_data['people_vaccinated_per_hundred'].iloc[0:50].tolist()
    #vaccine_state_count = filter_us_uhg_state_vaccine_data['people_vaccinated'].iloc[0:50].tolist()
    
    vaccine_state_dict = {}

    for i in range(0,len(vaccine_state),1):
        vaccine_state_dict[vaccine_state[i]] = vaccine_state_values[i]
        
    vaccine_state_dict = {k: v for k, v in sorted(vaccine_state_dict.items(),reverse = True, key=lambda x: x[1])[:5]} 
    
    expression = []
    
    for i in range(0, len(vaccine_state_dict),1):
        expression.append(list(vaccine_state_dict)[i])
        expression.append((list(vaccine_state_dict.values())[i]))
        #expression.append((list(vaccine_state_dict.values())[i][1]))
    
    print(expression)
    
    if len(vaccine_state_dict) == 0:
          a = ''
    
    elif len(vaccine_state_dict) == 1:
      
        a = 'Key UHG States with the highest vaccination rates: {} ({:.1f}%).'.format(*expression)
        
    
    elif len(vaccine_state_dict) == 2:
        
        a = 'Key UHG States with the highest vaccination rates: {} ({:.2f}%) and {} ({:.2f}%).'.format(*expression)
        
    elif len(vaccine_state_dict) > 2:
        
        comment_length = len(vaccine_state_dict) - 2
        comment = 'Key UHG States with the highest vaccination rates: ' + '{} ({:.1f}%)' + comment_length*(', {} ({:.1f}%)') + ' and {} ({:.1f}%)' + str("") + '.'.format(*expression)
        a = comment.format(*expression)
    return a    

  
def us_7day_ma(df):
   
    filter_cond = (df['Latest_Date_Rank'] == 1) & (df['location'] != "United States")
    
    filter_cond_pm = (df['Latest_Date_Rank'] == 29) & (df['location'] != "United States")
    
    filter_us_7day_ma_data = df[filter_cond]
    
    filter_us_7day_ma_data_pm = df[filter_cond_pm]
    
    #print(filter_us_7day_ma_data_pm)
    
    us_7day_ma = round(filter_us_7day_ma_data["US Daily Vaccinations 7 Day MA"].mean())
    
    us_7day_ma_pm = round(filter_us_7day_ma_data_pm["US Daily Vaccinations 7 Day MA"].mean())
    
    
    try:
        change_per =(us_7day_ma/us_7day_ma_pm - 1)*100
    except:
        return 0
    
    maxdt_cw = filter_us_7day_ma_data['date'].max()
       
    month_cw = maxdt_cw.month
    day_cw =maxdt_cw.day

    if(change_per > 0):
        flag = 'up'
    else:
        flag = 'down'
    
    a = '{:,.0f} nominal 7-day moving average of vaccine doses administered on {}/{}, {} {:.1f}% from {:,.0f} one month ago.'.format(us_7day_ma,month_cw,day_cw,flag,abs(change_per),us_7day_ma_pm) 
    
    return a
  
def US_Vaccines_Pop_Per(df):
    
    filter_cond = (df['Latest_Date_Rank'] == 1) & (df['location'] == "United States")
    
    filtered_vaccine_data = df[filter_cond]
    
    people_partially_vaccinated = (filtered_vaccine_data['people_vaccinated'].sum())
    
    people_fully_vaccinated = (filtered_vaccine_data['people_fully_vaccinated_per_hundred'].sum())
    
    population = (filtered_vaccine_data['Population'].sum())
    
    Population_Partially_Vaccinated = (people_partially_vaccinated/population) * 100
    
    Population_Fully_Vaccinated = people_fully_vaccinated

    a = '{:.1f}% of the population are partially vaccinated, with {:.1f}% of the population fully vaccinated.'.format(Population_Partially_Vaccinated,Population_Fully_Vaccinated) 

    return a  


US_Vaccines__Pop_Percetage = US_Vaccines_Pop_Per(us_vaccine_data)
print(US_Vaccines__Pop_Percetage)

USA_total_vaccine_dose_admin = usa_total_vaccine(us_vaccine_data)
print("----------USA_Total_Vaccine---------------")
print(USA_total_vaccine_dose_admin)

global_total_vaccine_dose_admin = global_total_vaccine(vaccine_data)
print("-----------Global_Total_Vaccine--------------")
print(global_total_vaccine_dose_admin)

global_7day_ma_vaccine = global_7day_ma(vaccine_data)
print("------------Global_7day_MA-------------")
print(global_7day_ma_vaccine)

global_UHG_location_vaccines = global_uhg_loc_vax(vaccine_data)
print("-------------------------")
print(global_UHG_location_vaccines) 


us_7day_ma_vaccine = us_7day_ma(cdc_vaccine_data)
print("-------------USA_7day_MA------------")
print(us_7day_ma_vaccine)
print("------------us_uhg_state_vaccines-------------")
us_uhg_state_vaccines = us_uhg_state_vaccines(cdc_vaccine_data,state_map)
print("-------------------------")
print(us_uhg_state_vaccines)

Global_Nominal_Vaccine_Measure = vaccine_nominal(vaccine_data,'World') 

USA_Nominal_Vaccine_Measure = vaccine_nominal(cdc_vaccine_data,'United States') 

print("-------------USA Nominal Vaccine------------")
print(USA_Nominal_Vaccine_Measure)
print("------------Global Nominal Vaccine-------------")
print(Global_Nominal_Vaccine_Measure)


51.5% of the population are partially vaccinated, with 41.2% of the population fully vaccinated.
----------USA_Total_Vaccine---------------
297,720,928 total vaccine doses administered as of 6/3.
-----------Global_Total_Vaccine--------------
188,354,235 total vaccine doses administered across 89 countries as of 2/17.
------------Global_7day_MA-------------
4,558,695 nominal 7-day moving average of vaccine doses administered on 2/17, up 51.7% from 3,004,444 one month ago.
-------------------------
The key UHG locations with the highest percentage of population vaccinated are: UK 23.5% (15,940,972), Chile 12.1% (2,320,697), U.S. 12.0% (40,268,009), Ireland 3.6% (180,192), Portugal 3.4% (347,013), Brazil 2.5% (5,326,007), India 0.7% (9,422,228), Peru 0.4% (118,457) and Colombia 0.0% (18).
-------------USA_7day_MA------------
1,660,754 nominal 7-day moving average of vaccine doses administered on 2/18, up 94.1% from 855,773 one month ago.
------------us_uhg_state_vaccines-------------
['CT

NameError: name 'vaccine_nominal' is not defined